In [1]:
import psutil
import platform
from datetime import datetime

In [2]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

In [3]:
print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

======================================== System Information ========================================
System: Windows
Node Name: DESKTOP-770U419
Release: 10
Version: 10.0.17763
Machine: AMD64
Processor: Intel64 Family 6 Model 158 Stepping 12, GenuineIntel


In [4]:
# Boot Time
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

======================================== Boot Time ========================================
Boot Time: 2019/10/9 21:3:35


In [5]:
# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 8
Total cores: 8
Max Frequency: 3600.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 3600.00Mhz
CPU Usage Per Core:
Core 0: 18.8%
Core 1: 5.5%
Core 2: 5.2%
Core 3: 10.4%
Core 4: 5.0%
Core 5: 2.3%
Core 6: 4.6%
Core 7: 4.6%
Total CPU Usage: 7.2%


In [6]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 31.94GB
Available: 25.40GB
Used: 6.54GB
Percentage: 20.5%
==================== SWAP ====================
Total: 36.69GB
Free: 28.12GB
Used: 8.57GB
Percentage: 23.4%


In [7]:
# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_size(partition_usage.total)}")
    print(f"  Used: {get_size(partition_usage.used)}")
    print(f"  Free: {get_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

======================================== Disk Information ========================================
Partitions and Usage:
=== Device: C:\ ===
  Mountpoint: C:\
  File system type: NTFS
  Total Size: 953.32GB
  Used: 226.61GB
  Free: 726.71GB
  Percentage: 23.8%
=== Device: D:\ ===
  Mountpoint: D:\
  File system type: NTFS
  Total Size: 931.50GB
  Used: 14.86GB
  Free: 916.64GB
  Percentage: 1.6%
Total read: 10.68GB
Total write: 1.31GB


In [8]:
# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

======================================== Network Information ========================================
=== Interface: Hamachi ===
=== Interface: Hamachi ===
  IP Address: 25.50.170.80
  Netmask: 255.0.0.0
  Broadcast IP: None
=== Interface: Hamachi ===
=== Interface: Hamachi ===
=== Interface: 이더넷 ===
=== Interface: 이더넷 ===
  IP Address: 183.100.235.22
  Netmask: 255.255.255.0
  Broadcast IP: None
=== Interface: 이더넷 ===
=== Interface: Wi-Fi ===
=== Interface: Wi-Fi ===
  IP Address: 169.254.64.149
  Netmask: 255.255.0.0
  Broadcast IP: None
=== Interface: Wi-Fi ===
=== Interface: 로컬 영역 연결* 1 ===
=== Interface: 로컬 영역 연결* 1 ===
  IP Address: 169.254.97.104
  Netmask: 255.255.0.0
  Broadcast IP: None
=== Interface: 로컬 영역 연결* 1 ===
=== Interface: 로컬 영역 연결* 2 ===
=== Interface: 로컬 영역 연결* 2 ===
  IP Address: 169.254.155.170
  Netmask: 255.255.0.0
  Broadcast IP: None
=== Interface: 로컬 영역 연결* 2 ===
=== Interface: Bluetooth 네트워크 연결 ===
=== Interface: Bluetooth 네트워크 연결 ===
  IP Address: 169.254.